In [ ]:
import pandas as pd

Let's import the OTU table and the associated mapping file

In [ ]:
OTUtable=pd.read_csv('fullabsOTUsl7GL4PAcopy.csv',index_col='#OTU ID')
metadata=pd.read_csv('mapflcategoricalcopy.csv',header=[0,1])

In [ ]:
#look at tables -type in

Because we imported the metadata table with a multi-index header, we need to set the row indices separately

In [ ]:
#set row index as "index"
metadata = metadata.set_index("index")

#it sets as a wierd tuple, so needs to be fixed
rind = []
for ind in metadata.index:
    rind.append(ind[0])
metadata.index = rind

Look at the metdata table to observe the multi-indexing. Notice that all of my metadata columns are ranked/categorical from prior data analysis except sample depth.

In [ ]:
metadata

Notice that i have not set an index column for metadata. This is because I am importing the dataframe with a multi-indexed header. The first row of headers indicates the type of data that the column header below is (index,continuous data, categorical data). 

We can slice along indices at varying levels by using the `metadata.xs` function. the syntax is `metadata.xs('index category',index level, axis)`. Let's examine the indexing a few different ways.

In [ ]:
#access all columns with the level 0 index "categorical"
metadata.xs('categorical',level=0,axis=1)

notice that I get all 4 columns of data in the dataframe that are labelled at indexing level 0 as "categorical"

In [ ]:
#access the column "sulfide" within that level
metadata.xs('categorical',level=0,axis=1)['sulfide']

Let's exame index level one. 

In [ ]:
metadata.xs('sulfide',level=1,axis=1)

Notice that the level 0 index is visible as a column header

The first thing to do is filter our OTU table. First let's get rid of any OTUs that are empty in all samples, or any samples that have no OTU read counts (i.e., failed to amplify)

To do this, we can use the pandas `.all` function. The syntax is `(criterion).all(axis)`, where `axis=1` is across all columns and `axis=0` is across all rows. 

In [ ]:
#show rows from OTU table that have na in all samples. 
OTUtable.loc[OTUtable.isna().all(axis=1)]

Notice that we only have an empty data table returned. This means there are no OTUs that need to be removed by this criterion. Now let's check the other way (are any samples all NAN?)

In [ ]:
#show columns from OTU table that have na across all OTUs. 
OTUtable.isna().all(axis=0)

The function returns each column header as an index with an associated boolean value. In this case, the blank has NaN for all OTUs. Let's retrieve the column headers of the columns we want to keep...where the condition is False (the boolean object's indices). 

In [ ]:
colsBoolobj=OTUtable.isna().all(axis=0)
keepCols=colsBoolobj.loc[colsBoolobj==False].index

Let's take a look at keepCols

In [ ]:
keepCols

In [ ]:
OTUtable=OTUtable[keepCols]

In [ ]:
OTUtable

Now that we haved removed NaN we need to further filter our OTU table.

The pandas `str.contains` function can be used to search entire columns for substrings. In this case, we are going to use `.index` to search the OTU ID column that we have set as the index column for the data frame for chloroplasts. To search for multiple substrings at once, they can be entered into the `str.contains` function as one string and separated by a `|`

In [ ]:
#show chloroplast entries in OTU table
OTUtable[OTUtable.index.str.contains('Chloroplast|chloroplast')]

Notice when we look at the output of this command, we get a pandas dataframe showing all the rows where **Chloroplast** or **chloroplast** is in the OTU ID. 

We can use the `~` operator to return a pandas dataframe that does NOT contain those rows

In [ ]:
#filter chloroplasts out of OTU table
OTUtable=OTUtable[~OTUtable.index.str.contains('Chloroplast|chloroplast')]

Now we can filter the data based on any other criteria we choose.

The same syntax can be used as above in the pandas function `.isna()` to look for OTUs with less than 10 reads in all samples. We can use `.all` in combination with `.loc` to refer to row index. 

In [ ]:
#show rows from OTU table that have less than 10 reads in all samples
OTUtable.loc[(OTUtable<10).all(axis=1)]

In [ ]:
#remove rows from OTU table that have less than 10 reads in all samples
OTUtable=OTUtable.loc[~(OTUtable<10).all(axis=1)]

Finally, let's use the `str.contains` function one more time to filter the OTU table to only contain photoautotrophs

In [ ]:
photoautoOTUs=OTUtable[OTUtable.index.str.contains('Cyanobiaceae|Chlorobiaceae|Chromatiaceae')]

In [ ]:
photoautoOTUs

Now we are ready to set up our metadata file. This OTU table is a subset of all samples, so we can use the pandas `.drop` function to remove rows from the metadata table that are irrelevant. In the meanwhile, we can also retrieve a list of the appropriate indices to use later.

In [ ]:
toRem=[]

for s in metadata.index:
    print(s)
    if s not in photoautoOTUs.columns:
        toRem.append(s)

In [ ]:
toRem

In [ ]:
clMetadata=metadata.drop(toRem)

let's look at our cleaned metadata table

In [ ]:
clMetadata

In [ ]:
#examine categorical data
clMetadata.xs('categorical',level=0,axis=1)

We can take advantage of our multi-indexing along with the `.groupby` function to exame our photoautotroph OTUs in relationship to our categorical metadata columns. First, we need to reformat the OTU table to match the metadata table. This can be done with the pandas `transpose` function.

In [ ]:
photoautoOTUsT=photoautoOTUs.T

In [ ]:
for OTU in photoautoOTUsT.columns:
    print("OTU: ",OTU)
    temp=pd.concat([photoautoOTUsT[OTU],clMetadata.xs('categorical',level=0,axis=1)],axis=1)
    for c in temp.columns[1:]:        
        for name, group in temp.groupby([c]):
            print(c, ":", name)
            print(group)
            print("\n\n\n")